In [6]:
import requests
import json
import pandas as pd
import re

In [ ]:
import configparser
import os

config = configparser.ConfigParser()

# Ścieżka do pliku konfiguracyjnego
config_file_path = os.path.join(os.getcwd(), "config.ini")
config.read(config_file_path)

try:
    OPENROUTER_API_KEY = config['API']['OPENROUTER_API_KEY']
    PATH_SICK = config['Paths']['PATH_SICK']
    PATH_CONTROL = config['Paths']['PATH_CONTROL']
    MODEL_1 = config['Models']['MODEL_1']
    MODEL_2 = config['Models']['MODEL_2']
    print("Plik konfiguracyjny pomyślnie wczytany.")
except KeyError as e:
    print(f"Błąd: Brak klucza konfiguracyjnego: {e}\nUżywam ustawień domyślnych.")
    OPENROUTER_API_KEY = None
    # Ustawienia domyślne, dla jakich tworzono ten skrypt
    PATH_SICK = "data/descriptions_sick.xlsx"
    PATH_CONTROL = "data/descriptions_control_group.xlsx"
    MODEL_1 = "google/gemini-2.0-flash-exp:free"
    MODEL_2 = "deepseek/deepseek-chat-v3-0324:free"

Plik konfiguracyjny pomyślnie wczytany.


In [ ]:
#Otwórz pliki Excel z danymi
controlGroup = pd.read_excel(PATH_CONTROL)
sick = pd.read_excel(PATH_SICK)

#Sprawdzenie wczytanych danych
controlGroup.head(), sick.head()

(  patient_id                                        description
 0    PAT-123  I. Wywiad medyczny\n\nPacjent zgłosił się na w...
 1    PAT-124  I. Wywiad medyczny\n\nPacjent zgłosił się na k...
 2    PAT-125  I. Wywiad medyczny\n\nPacjent zgłasza się na k...
 3    PAT-126  Pacjent zgłosił się na rutynową kontrolę zdrow...
 4    PAT-127   Wywiad medyczny\n\nPacjent zgłosił się do lek...,
   patient_id                                        description
 0   PAT-9882  I. Wywiad medyczny\nPacjent: Mężczyzna/Kobieta...
 1   PAT-9661  I. Wywiad medyczny\nPacjent zgłosił się do por...
 2   PAT-7561  \n**Pacjent ID**: PAT-7561\n**Data wizyty**: [...
 3   PAT-5055  I. Wywiad medycznyPacjent: Mężczyzna/Kobieta, ...
 4   PAT-1026  \n\n### I. Wywiad medyczny\nPacjent zgłosił si...)

In [4]:
#Odczytaj plik z opisami zmiennych
with open("data/symptoms_description.txt", "r", encoding="utf-8") as file:
    columns_string = file.read()

In [7]:
#inicjalizacja pustej tabeli do przechowywania przekształconych danych

#uzyskanie nazw kolumn z pliku tekstowego
columns = [line.split(": ")[1] for line in columns_string.strip().split("\n")]
columns.insert(0, "ID")

#Utworzenie pustej tabeli, której wiersze będą generowane w pętli
sick_tr = pd.DataFrame(columns=columns)
controlGroup_tr = pd.DataFrame(columns=columns)
sick_tr

,ID,OS,OW,BB,NDN,DT,PC,PK,AL,NG,UF,IW,LK,WS,DS


In [ ]:
def generuj_tabele(data, transformed, key):
    i = len(transformed)

    while i < len(data):
        # Przygotowanie promptu dla modelu
        prompt_detect = f"""
        Jesteś ekspertem w diagnostyce dziedzicznego obrzęku naczynioruchowego (HAE). Oto opis pacjenta o ID {data["patient_id"][i]}:
        {data["description"][i]}

        Które z podanych objawów są obecne u pacjenta? Proszę odpowiedzieć tylko 1 (jeśli objaw występuje) lub 0 (jeśli objaw nie występuje) na każde pytanie. Nie podawaj żadnych dodatkowych informacji ani wyjaśnień. Oto lista objawów:
        obrzęki skóry: OS,
        obrzęki narządów wewnętrznych: OW,
        bóle brzucha: BB,
        nudności i wymioty: NDN,
        drętwienie: DT,
        pieczenie: PC,
        pokrzywka lub świąd: PK,
        alergia: AL,
        nagły atak obrzęków: NG,
        uraz fizyczny: UF,
        infekcja wirusowa: IW,
        nieskuteczność leków alergicznych: LK,
        występowanie podobnych objawów w rodzinie: WS,
        duszności: DS.

        Dostosuj odpowiedź do poniższego formatu:
        <skrót objawu>: <0 lub 1>
        Na przykład:
        OS: 1
        OW: 0
        BB: 1
        Proszę nie dodawać żadnych dodatkowych informacji ani wyjaśnień.
        """

        # Wysłanie zapytania do OpenRouter
        headers = {
            "Authorization": f"Bearer {key}",
            "Content-Type": "application/json"
        }

        payload = {
            "model": MODEL_2,
            "messages": [{"role": "user", "content": prompt_detect}],
            "temperature": 0.1,  # Kontrola kreatywności (0 = bardziej deterministyczne, 1 = bardziej losowe)
        }

        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=payload
        )
        result = response.json()

        if response.status_code == 200:
            if "choices" in result:
                numeric_values = [int(num) for num in re.findall(r'\d+', result["choices"][0]["message"]["content"])]
                numeric_values.insert(0, data["patient_id"][i])

                if len(numeric_values) != len(transformed.columns):
                    print(f"Wiersz {i}: Powinno być {len(transformed.columns)} wartości, ale jest {len(numeric_values)} w wygenerowanym wierszu. Powtarzanie...")
                else:
                    transformed.loc[len(transformed)] = numeric_values
                    print(f"Wiersz {i} został dodany do tabeli.")
                    i += 1
                
            elif "error" in result:
                print(f"Błąd: {result['error']['message']}")
                break
            else:
                print("Nieoczekiwany format odpowiedzi:", result)
                break
        else:
            print(f"Błąd: {response.status_code}, {response.text}")
            break

    return transformed

In [35]:
sick_tr = generuj_tabele(sick, sick_tr, OPENROUTER_API_KEY)

Wiersz 8 został dodany do tabeli.
Wiersz 9 został dodany do tabeli.
Wiersz 10 został dodany do tabeli.
Wiersz 11 został dodany do tabeli.
Wiersz 12 został dodany do tabeli.
Wiersz 13 został dodany do tabeli.
Wiersz 14 został dodany do tabeli.
Wiersz 15 został dodany do tabeli.
Wiersz 16 został dodany do tabeli.
Wiersz 17 został dodany do tabeli.
Wiersz 18 został dodany do tabeli.
Wiersz 19 został dodany do tabeli.
Wiersz 20 został dodany do tabeli.
Wiersz 21 został dodany do tabeli.
Wiersz 22 został dodany do tabeli.
Wiersz 23 został dodany do tabeli.
Wiersz 24 został dodany do tabeli.


In [36]:
sick_tr

,ID,OS,OW,BB,NDN,DT,PC,PK,AL,NG,UF,IW,LK,WS,DS
0,PAT-9882,1,1,1,0,0,0,0,0,1,1,1,1,1,0
1,PAT-9661,1,0,0,0,0,0,1,1,0,0,0,0,0,0
2,PAT-7561,1,0,0,0,0,0,0,0,1,1,1,1,1,0
3,PAT-5055,1,1,1,0,0,0,0,0,1,1,1,1,1,0
4,PAT-1026,0,1,0,0,0,0,0,0,1,1,1,0,1,1
5,PAT-3066,1,1,1,0,0,0,0,0,1,0,0,1,1,0
6,PAT-2784,1,0,0,0,0,0,0,0,1,1,1,0,1,0
7,PAT-4795,1,1,1,0,0,0,0,0,1,0,0,1,1,0
8,PAT-7996,1,1,0,0,0,0,0,0,1,1,1,0,1,1
9,PAT-3090,1,0,0,0,0,0,0,0,1,1,1,0,1,1


In [43]:
controlGroup_tr = generuj_tabele(controlGroup, controlGroup_tr, OPENROUTER_API_KEY)

Wiersz 67 został dodany do tabeli.
Wiersz 68 został dodany do tabeli.
Wiersz 69 został dodany do tabeli.
Wiersz 70 został dodany do tabeli.
Wiersz 71 został dodany do tabeli.
Wiersz 72 został dodany do tabeli.
Wiersz 73 został dodany do tabeli.
Wiersz 74 został dodany do tabeli.
Wiersz 75 został dodany do tabeli.
Wiersz 76 został dodany do tabeli.
Wiersz 77 został dodany do tabeli.
Wiersz 78 został dodany do tabeli.
Wiersz 79 został dodany do tabeli.
Wiersz 80 został dodany do tabeli.
Wiersz 81 został dodany do tabeli.
Wiersz 82 został dodany do tabeli.
Wiersz 83 został dodany do tabeli.
Wiersz 84 został dodany do tabeli.
Wiersz 85 został dodany do tabeli.
Wiersz 86 został dodany do tabeli.
Wiersz 87: Powinno być 15 wartości, ale jest 2 w wygenerowanym wierszu. Powtarzanie...
Wiersz 87 został dodany do tabeli.
Wiersz 88: Powinno być 15 wartości, ale jest 1 w wygenerowanym wierszu. Powtarzanie...
Wiersz 88 został dodany do tabeli.
Wiersz 89: Powinno być 15 wartości, ale jest 1 w wygener

In [44]:
controlGroup_tr.tail()

,ID,OS,OW,BB,NDN,DT,PC,PK,AL,NG,UF,IW,LK,WS,DS
95,PAT-218,0,0,1,0,0,0,0,0,0,0,0,0,0,0
96,PAT-219,0,1,1,0,0,0,0,0,0,0,0,0,0,0
97,PAT-220,0,0,0,0,0,0,0,0,0,1,0,0,0,0
98,PAT-221,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99,PAT-222,0,1,1,0,0,0,0,0,0,1,0,0,0,0


In [47]:
sick_tr["HAE"] = 1
sick_tr.head()

,ID,OS,OW,BB,NDN,DT,PC,PK,AL,NG,UF,IW,LK,WS,DS,HAE
0,PAT-9882,1,1,1,0,0,0,0,0,1,1,1,1,1,0,1
1,PAT-9661,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1
2,PAT-7561,1,0,0,0,0,0,0,0,1,1,1,1,1,0,1
3,PAT-5055,1,1,1,0,0,0,0,0,1,1,1,1,1,0,1
4,PAT-1026,0,1,0,0,0,0,0,0,1,1,1,0,1,1,1


In [45]:
controlGroup_tr["HAE"] = 0
controlGroup_tr.head()

,ID,OS,OW,BB,NDN,DT,PC,PK,AL,NG,UF,IW,LK,WS,DS,HAE
0,PAT-123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,PAT-124,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,PAT-125,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,PAT-126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,PAT-127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
disease_symptoms = pd.concat([sick_tr, controlGroup_tr], ignore_index=True)
disease_symptoms.to_csv("results/disease_symptoms.csv", index=False)
print("Tabela została zapisana do pliku 'results/disease_symptoms.csv'.")
# Sprawdzenie, czy tabela została poprawnie zapisana
disease_symptoms = pd.read_csv("results/disease_symptoms.csv")
# Wyświetlenie pierwszych kilku wierszy tabeli
disease_symptoms.head()

Tabela została zapisana do pliku 'dane/table_gemini.csv'.


,ID,OS,OW,BB,NDN,DT,PC,PK,AL,NG,UF,IW,LK,WS,DS,HAE
0,PAT-9882,1,1,1,0,0,0,0,0,1,1,1,1,1,0,1
1,PAT-9661,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1
2,PAT-7561,1,0,0,0,0,0,0,0,1,1,1,1,1,0,1
3,PAT-5055,1,1,1,0,0,0,0,0,1,1,1,1,1,0,1
4,PAT-1026,0,1,0,0,0,0,0,0,1,1,1,0,1,1,1
